In [5]:
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset

# Example data
users = [1, 2, 3, 4]
items = [1, 2, 3, 4, 5]
interactions = [
    (1, 1),
    (1, 2),
    (2, 2),
    (2, 3),
    (3, 3),
    (3, 4),
    (4, 4),
    (4, 5)
]

user_features = {
    1: [0, 1],  # User 1 has features 0 and 1
    2: [1, 2],  # User 2 has features 1 and 2
    3: [0, 2],  # User 3 has features 0 and 2
    4: [1, 3]   # User 4 has features 1 and 3
}

item_features = {
    1: [0, 1],  # Item 1 has features 0 and 1
    2: [1],     # Item 2 has feature 1
    3: [2],     # Item 3 has feature 2
    4: [0, 2],  # Item 4 has features 0 and 2
    5: [1, 3]   # Item 5 has features 1 and 3
}

# Convert the features to the appropriate format
user_features_list = [(user, [f"feature_{f}" for f in features]) for user, features in user_features.items()]
item_features_list = [(item, [f"feature_{f}" for f in features]) for item, features in item_features.items()]


In [6]:
# Initialize the dataset
dataset = Dataset()

# Fit the dataset with user and item ids and features
dataset.fit(
    users=users,
    items=items,
    user_features=[f"feature_{f}" for f in range(4)],
    item_features=[f"feature_{f}" for f in range(4)]
)


In [7]:
# Build the interactions matrix
(interactions, weights) = dataset.build_interactions(interactions)

# Build the user and item features matrices
user_features_matrix = dataset.build_user_features(user_features_list)
item_features_matrix = dataset.build_item_features(item_features_list)


In [11]:
# Initialize and train the model
latent_feats = 10
training_epochs = 30

model = LightFM(loss='warp', no_components=latent_feats)
model.fit(interactions, user_features=user_features_matrix, item_features=item_features_matrix, epochs=training_epochs, num_threads=8)


In [12]:
# Predicting the score for a user-item pair
user_id = 1
item_id = 3
score = model.predict(np.array([user_id]), np.array([item_id]), user_features=user_features_matrix, item_features=item_features_matrix)
print(f"Predicted score for user {user_id} and item {item_id}: {score[0]}")

# Evaluating the model
from lightfm.evaluation import precision_at_k, auc_score

train_precision = precision_at_k(model, interactions, k=5, user_features=user_features_matrix, item_features=item_features_matrix).mean()
train_auc = auc_score(model, interactions, user_features=user_features_matrix, item_features=item_features_matrix).mean()

print(f'Train precision at k: {train_precision:.2f}')
print(f'Train AUC score: {train_auc:.2f}')


Predicted score for user 1 and item 3: -0.6869043111801147
Train precision at k: 0.40
Train AUC score: 0.62
